<a href="https://colab.research.google.com/github/hannibal216/colab_practice/blob/main/tensorflow_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test notebook taken from [Google Colab Tutorial](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb#scrollTo=-47O6_GLdRuT)

## Set up TensorFlow

In [6]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.7.0


## Load a dataset

In [7]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model
Build a tf.keras.Sequential model by stacking layers.

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [9]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.02874227, -0.20210332,  0.9114835 ,  1.2130837 , -0.2761817 ,
         0.3258256 , -0.2715602 ,  0.10015875,  0.30534592,  0.42859477]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.06680828, 0.05617467, 0.17106658, 0.23128554, 0.05216374,
        0.09523951, 0.05240537, 0.0759996 , 0.09330887, 0.10554782]],
      dtype=float32)


Note: It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [12]:
loss_fn(y_train[:1], predictions).numpy()

2.3513603

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [14]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model
Use the Model.fit method to adjust your model parameters and minimize the loss:

In [15]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2892 - accuracy: 0.9165
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1398 - accuracy: 0.9579
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1057 - accuracy: 0.9681
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0860 - accuracy: 0.9739
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0750 - accuracy: 0.9768



The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [17]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0743 - accuracy: 0.9762 - 467ms/epoch - 1ms/step


[0.07434530556201935, 0.9761999845504761]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [18]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [19]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.5016473e-08, 1.5704984e-08, 3.2459254e-06, 1.4960327e-06,
        6.9783784e-10, 5.6939382e-09, 1.6686351e-13, 9.9998105e-01,
        7.9381323e-08, 1.4079772e-05],
       [1.7399598e-08, 2.0761188e-05, 9.9997199e-01, 3.1391883e-06,
        6.1491529e-13, 1.1215121e-06, 8.6188767e-10, 2.6594579e-13,
        2.9272537e-06, 5.3459275e-11],
       [2.0092311e-07, 9.9945647e-01, 5.5977336e-05, 4.4772582e-06,
        2.1297825e-05, 9.3221561e-06, 2.0277901e-05, 3.4798530e-04,
        8.3094841e-05, 1.0480107e-06],
       [9.9958569e-01, 5.3599177e-08, 4.8695241e-05, 1.4059654e-08,
        7.4546633e-06, 1.1899631e-07, 3.2382898e-04, 3.9454862e-06,
        6.2920236e-07, 2.9496368e-05],
       [4.9879354e-06, 7.6304927e-09, 1.7601356e-05, 8.4378762e-09,
        9.9454367e-01, 8.9326721e-07, 7.3045571e-06, 1.8561234e-04,
        7.7728544e-07, 5.2391803e-03]], dtype=float32)>